<a href="https://colab.research.google.com/github/ajmainrahman/ML-Projects/blob/main/Ginger_and_Garlic_using_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imorts

In [ ]:
import tensorflow as tf
import os
from functools import partial
from collections import namedtuple
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle

from tensorflow.keras.applications.efficientnet import EfficientNetB7
from tensorflow.keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications import ResNet50

from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.applications import ResNet101V2
from tensorflow.keras.applications import ResNet152

from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications import MobileNetV2


from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.layers import Conv2D, MaxPool2D, Activation, Dropout, BatchNormalization

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
from tensorflow.keras.optimizers import SGD

# Prepare the files

In [ ]:
#INPUT_ROOT = '/content/drive/MyDrive/lung and colon cancer/lung_colon_image_set'
#DATA_DIR = os.path.join(INPUT_ROOT, '/content/drive/MyDrive/lung and colon cancer/lung_colon_image_set/colon_image_sets', '/content/drive/MyDrive/lung and colon cancer/lung_colon_image_set/lung_image_sets')

In [ ]:
INPUT_ROOT = '/content/drive/MyDrive/Dataset/GarlicGingerDataset/Train'
DATA_DIR = os.path.join(INPUT_ROOT, '/content/drive/MyDrive/Research Paper - 22/Research with Asad Sir/Dataset/Lung_and_Colon_Cancer/Colon_Cancer','/content/drive/MyDrive/Research Paper - 22/Research with Asad Sir/Dataset/Lung_and_Colon_Cancer/Lung_Cancer')

In [ ]:
train_dir = "/content/drive/MyDrive/Dataset/GarlicGingerDataset/Train"   #Setting training directory
validation_dir = "/content/drive/MyDrive/Dataset/GarlicGingerDataset/Test"   #Setting testing directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator 

# Generate the data

In [ ]:
data = image.ImageDataGenerator(validation_split = 0.2)
BATCH_SIZE = 128
# Change batch size, if you run out of memory!
# Or increase it, if you have a lot of RAM (>> 16 Gb) for faster trainig
IMG_SHAPE = (256, 256, 3)

In [ ]:
generate_data = partial(
    data.flow_from_directory,
    validation_dir,
    class_mode = 'categorical',
    color_mode = 'rgb',
    target_size = IMG_SHAPE[:2],
    batch_size = BATCH_SIZE,
    shuffle = False,
    seed = 666
)

print('Trainig data:')
train_iter = generate_data(subset='training')
print('Validating data:')
validate_iter = generate_data(subset='validation')

Trainig data:


FileNotFoundError: ignored

In [ ]:
print(generate_data)

functools.partial(<bound method ImageDataGenerator.flow_from_directory of <keras.preprocessing.image.ImageDataGenerator object at 0x7f6340ef7510>>, '/content/drive/MyDrive/Research Paper - 22/Research with Asad Sir/Dataset/Lung_and_Colon_Cancer/Lung_Cancer', class_mode='categorical', color_mode='rgb', target_size=(256, 256), batch_size=128, shuffle=False, seed=666)


In [ ]:
print(IMG_SHAPE[:2])

(256, 256)


In [ ]:
import numpy as np
l = list(train_iter[0])
print(l[0].shape)
print(l[1].shape)

(128, 256, 256, 3)
(128, 3)


In [ ]:
print(len(l[1]))

128


In [ ]:
'''
for i in range(15):
    print(l[0][i])
    print("\n")
'''    
arr = l[0][1]
print(type(arr))

<class 'numpy.ndarray'>


In [ ]:
from PIL import Image as im
data = im.fromarray(arr,'RGB')
data.save('gfg_dummy_pic.jpeg')

In [ ]:
CLASSES_NUM = 3
ACCURACY_THRESHOLD = 0.99

# Set to True to train simple from scratch CNN
RUN_SIMPLE_CNN = False

# Set to True to use fine-tuning
RUN_PRETRAINED = True

# Fine-tuned models

Some constants that are used for this particular dataset:

### Preparing pretrained models

In [ ]:
PreparedModel = namedtuple('PreparedModel', ['name', 'keras_pretrained', 'optimizer'])

In [ ]:
def get_keras_pretrained(keras_model):
    pretrained = keras_model(
        input_shape=IMG_SHAPE,
        weights='imagenet',
        include_top=False
    )
    pretrained.trainable = False
    return pretrained

In [ ]:
if RUN_PRETRAINED:
    prepared_VGG16 = PreparedModel('VGG16', get_keras_pretrained(VGG16), 'adam')
    prepared_EfficientNetB7 = PreparedModel('EfficientNetB7', get_keras_pretrained(EfficientNetB7), 'adam')
    prepared_InceptionV3 = PreparedModel('InceptionV3', get_keras_pretrained(InceptionV3), SGD(learning_rate=0.001, momentum=0.9))
    prepared_ResNet50 = PreparedModel('ResNet50', get_keras_pretrained(ResNet50), 'adam')
  
    prepared_VGG19 = PreparedModel('VGG19', get_keras_pretrained(VGG19), 'adam')
    prepared_ResNet50V2 = PreparedModel('ResNet50V2', get_keras_pretrained(ResNet50V2), 'adam')
  
    
    #prepared_InceptionResNetV2 = PreparedModel('InceptionResNetV2', get_keras_pretrained(InceptionResNetV2), SGD(learning_rate=0.001, momentum=0.9))
    #prepared_MobileNet = PreparedModel('MobileNet', get_keras_pretrained(MobileNet), 'adam')
    #prepared_MobileNetV2 = PreparedModel('MobileNetV2', get_keras_pretrained(MobileNetV2), 'adam')  
    

    # If you have no time limit, you can include all the models and run it
    # But if you use kaggle, it's better to include just one and run multiple times
    # You can also use the callback below or EarlyStopping
    #models = [prepared_ResNet101, prepared_ResNet101V2, prepared_ResNet152, prepared_ResNet152V2, prepared_InceptionResNetV2, prepared_MobileNet, prepared_MobileNetV2, prepared_DenseNet121, prepared_DenseNet169, prepared_DenseNet201]
    models = [prepared_EfficientNetB7]

In [ ]:
class TresholdStopping(tf.keras.callbacks.Callback):
    
    def __init__(self, point):
        super(TresholdStopping, self).__init__()
        self.point = point

    def on_epoch_end(self, epoch, logs=None): 
        accuracy = logs['val_accuracy']
        if accuracy >= self.point:
            self.model.stop_training = True

treshold_stopping = TresholdStopping(ACCURACY_THRESHOLD)

### Fit using our top layer

In [ ]:
def top_layer(output):
    layer = GlobalAveragePooling2D()(output)
    layer = Flatten()(layer)
    layer = Dense(128, activation='relu')(layer)
    # layer = Dropout(0.5)(layer)
    # Drop out layer is optional: you may include it to fix overfitting
    layer = Dense(64, activation='relu')(layer)
    layer = Dense(CLASSES_NUM, activation='softmax')(layer)
    return layer

Fit the models, saving weigths and logs

In [ ]:
if RUN_PRETRAINED:    
    for prepared_model in models:

        keras_pretrained = prepared_model.keras_pretrained

        model = Model(inputs=keras_pretrained.input, outputs=top_layer(keras_pretrained.output))

        model.compile(
          loss='categorical_crossentropy',
          optimizer=prepared_model.optimizer,
          metrics=['accuracy'],

        )

        checkpointing = ModelCheckpoint(
            filepath=prepared_model.name,
            save_weights_only=True,
            save_best_only=True,
            monitor='val_accuracy',
            mode='max',
            verbose=1
        )

        log_saving = CSVLogger(f'{prepared_model.name}-log.csv', append=True, separator=';')

        print(f'STARTED TRAINIG OF {prepared_model.name} PREPARED MODEL')

        model.fit(
            train_iter,
            validation_data=validate_iter,
            epochs=20,
            callbacks=[checkpointing, log_saving]
        )
        
        # save model
        filename = 'finalized_model.sav'
        pickle.dump(model, open(filename, 'wb'))
        
        model.save('finalized_model.h5')
        
        print(f'TRAINIG OF {prepared_model.name} COMPLETED')
        print('------------------------------')

STARTED TRAINIG OF EfficientNetB7 PREPARED MODEL
Epoch 1/20
94/94 [==============================] - ETA: 0s - loss: 0.4682 - accuracy: 0.7956 
Epoch 1: val_accuracy improved from -inf to 0.94270, saving model to EfficientNetB7
94/94 [==============================] - 2366s 25s/step - loss: 0.4682 - accuracy: 0.7956 - val_loss: 0.1619 - val_accuracy: 0.9427
Epoch 2/20
94/94 [==============================] - ETA: 0s - loss: 0.1624 - accuracy: 0.9380
Epoch 2: val_accuracy did not improve from 0.94270
94/94 [==============================] - 209s 2s/step - loss: 0.1624 - accuracy: 0.9380 - val_loss: 0.1847 - val_accuracy: 0.9234
Epoch 3/20
94/94 [==============================] - ETA: 0s - loss: 0.1344 - accuracy: 0.9478
Epoch 3: val_accuracy improved from 0.94270 to 0.95536, saving model to EfficientNetB7
94/94 [==============================] - 210s 2s/step - loss: 0.1344 - accuracy: 0.9478 - val_loss: 0.1136 - val_accuracy: 0.9554
Epoch 4/20
94/94 [==============================] - ET

TRAINIG OF EfficientNetB7 COMPLETED
------------------------------


In [ ]:
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))
#model.save('finalized_model.h5')
#model.save_weights('my_model_weights.h5')

In [ ]:
print(train_iter)

# Model from scratch

To compare with fine-tuning approach, we will also train simple CNN model without any typical architecture and weights.

In [ ]:
# Features extraction layers
simple_CNN_model = tf.keras.models.Sequential([
    
    Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=IMG_SHAPE),
    BatchNormalization(),
    MaxPool2D(2),
    
    Conv2D(64, (3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    MaxPool2D(2),
  
    Conv2D(128, (3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    MaxPool2D(2)
])

# Classification layers
simple_CNN_model.add(Flatten())
simple_CNN_model.add(Dense(128, activation='relu'))
simple_CNN_model.add(Dense(64, activation='relu'))
simple_CNN_model.add(Dense(CLASSES_NUM, activation='softmax'))

In [ ]:
simple_CNN_model.compile(loss = 'categorical_crossentropy', optimizer = 'adam',metrics=['accuracy'])

Callbacks for saving weigths and log:

In [ ]:
name = 'SimpleCNN'

checkpointing = ModelCheckpoint(
    name,
    save_weights_only=True,
    save_best_only=True,
    monitor='val_accuracy',
    mode='max',
    verbose=1
)

log_saving = CSVLogger(f'{name}-log.csv', append=True, separator=';')

In [ ]:
if RUN_SIMPLE_CNN:
    
    print(f'{name} IS READY FOR TRAINIG:')
    simple_CNN_model.summary()
    
    print(f'STARTED TRAINIG OF {name} FROM SCRATCH MODEL')

    simple_CNN_model.fit(
        train_iter,
        validation_data=validate_iter,
        epochs=1,
        callbacks=[checkpointing, log_saving]
    )

    print(f'TRAINIG OF {name} COMPLETED')
    print('------------------------------')

In [ ]:
from PIL import Image
img = Image.open('/content/drive/MyDrive/lung and colon cancer/lung_colon_image_set/lung_image_sets/lung_aca/lungaca1.jpeg')

In [ ]:
#eff = pickle.load(open('../input/finalized-model/finalized_model.sav', 'rb'))
import h5py
from keras import models


In [ ]:
model = models.load_model('/content/finalized_model.h5')

In [ ]:
X_test = Image.open('/content/drive/MyDrive/lung and colon cancer/lung_colon_image_set/lung_image_sets/lung_aca/lungaca1.jpeg')

from sklearn.metrics import accuracy_score
#y_pred = model.predict(X_test)
#print(y_pred)

In [ ]:
from keras.models import load_model
from keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np
import os
import keras
import tensorflow as tf


def load_image(img_path, show=False):

   img = keras.utils.load_img(img_path, target_size=(256, 256,3))
    #img = image.load_img(img_path, target_size=(256, 256,3))
    img_tensor = image.img_to_array(img)                    # (height, width, channels)
    img_tensor = np.expand_dims(img_tensor, axis=0)         # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
    #img_tensor /= 255.                                      # imshow expects values in the range [0, 1]

    if show:
        plt.imshow(img_tensor[0])                           
        plt.axis('off')
        plt.show()

    return img_tensor


# load model
#model = load_model("model_aug.h5")


img_path1 = img_path+'lung_aca/lungaca10.jpeg'
img_path2 = img_path+'lung_n/lungn1000.jpeg'
img_path3 = img_path+'lung_scc/lungscc1.jpeg'
# load a single image
new_image = load_image(img_path1)

# check prediction
pred = model.predict(new_image)
print(pred)

In [ ]:
print(pred)

In [ ]:
print(pred)

In [ ]:
# plotting the loss
plt.plot(history.history['loss'],label = 'train_loss')
plt.plot(history.history['val_loss'], label = 'testing_loss')
plt.title('loss')
plt.legend()
plt.show()

In [ ]:
# Both Validation and Training accuracy is shown here

plt.plot(history.history['accuracy'], label='training_accuracy')
plt.plot(history.history['val_accuracy'], label='validation accuracy')
plt.title('Accuracy')
plt.legend()
plt.show()

In [ ]:
# CHECKING THE CONFUSION MATRIX

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
Y_pred = model.predict_generator(validate_set)
y_pred = np.argmax(Y_pred ,axis =1)
print('Confusion Matrix')
confusion_matrix = confusion_matrix(validate_set.classes, y_pred)
print(confusion_matrix)
print('Classification Report')
target_names = ['aca','n', 'scc']
print(classification_report(validate_set.classes, y_pred, target_names=target_names))